In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

from sph_lib.deposition import p2g
from sph_lib.utils import (compute_hsm, 
                           compute_hsm_tensor, 
                           project_hsm_tensor_to_2d
                           )                                

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
dims         = [2, 3]
datasets     = ['random', 'cosmo']
gridnum      = 100
num_neigh    = 24

In [3]:
method       = 'isotropic'
for dim in dims:
    print("-" * 50)
    print(f"Method: {method} {dim}D")

    for dataset in datasets:
        with open(f'../../../data/dataset_{dataset}_{dim}d.pkl', 'rb') as f:
            data = pickle.load(f)
        pos = data['pos'][:]
        mass = data['mass'][:, np.newaxis]
        total_mass_true = mass.sum()
        boxsize = data['boxsize']
        extent=[0, boxsize]

        hsm = compute_hsm(
            pos,
            nn=num_neigh,
            boxsize=boxsize
            #TODO: periodicity argument missing??
            )[0]
        kwargs = {'positions': pos, 
                    'quantities': mass, 
                    'hsm': hsm, 
                    'extent': extent,
                    'gridnum': gridnum,
                    'periodic': True
                    }
        
        res = p2g(
            **kwargs,
            averaged=[False] * mass.shape[1],
            method=method,
            use_python=False,
            )
        total_mass_deposited = res.sum()
            
        print(f"Total mass error on {dataset}:\t", 100 * np.round(abs(total_mass_deposited - total_mass_true) / total_mass_true, 5) if total_mass_deposited is not None else "N/A", "%")

--------------------------------------------------
Method: isotropic 2D
Using deposition function: isotropic_2d
Total mass error on random:	 0.11100000000000002 %
Using deposition function: isotropic_2d
Total mass error on cosmo:	 0.88299996 %
--------------------------------------------------
Method: isotropic 3D
Using deposition function: isotropic_3d
Total mass error on random:	 0.27799999999999997 %
Using deposition function: isotropic_3d
Total mass error on cosmo:	 1.546 %


In [6]:
method = 'anisotropic'
for dim in dims:
    print("-" * 50)
    print(f"Method: {method} {dim}D")

    for dataset in datasets:
        with open(f'../../../data/dataset_{dataset}_{dim}d.pkl', 'rb') as f:
            data = pickle.load(f)
        pos = data['pos'][:]
        mass = data['mass'][:, np.newaxis]
        total_mass_true = mass.sum()
        boxsize = data['boxsize']
        extent=[0, boxsize]

        hsm_tensor, hsm_eigvals, hsm_eigvecs = compute_hsm_tensor(
            pos,
            mass,
            NN=num_neigh,
            boxsize=boxsize
            )
        print(hsm_tensor.shape)
        if dim == 3:
            hsm_tensor, hsm_eigvals, hsm_eigvecs = project_hsm_tensor_to_2d(
                hsm_tensor,
                plane=(0, 1)
            )
        
        res = p2g(
            positions=pos,
            quantities=mass,
            hmat_eigvals=hsm_eigvals,
            hmat_eigvecs=hsm_eigvecs,
            extent=extent,
            gridnum=gridnum,
            periodic=True,
            averaged=[False] * mass.shape[1],
            method=method,
            use_python=False,
            )
        total_mass_deposited = res.sum()
            
        print(f"Total mass error on {dataset}:\t", 100 * np.round(abs(total_mass_deposited - total_mass_true) / total_mass_true, 4) if total_mass_deposited is not None else "N/A", "%")

--------------------------------------------------
Method: anisotropic 2D
(1000000, 24, 2) (1000000, 24)
(1000000, 2, 2)
Using deposition function: anisotropic_2d
Total mass error on random:	 0.13999999999999999 %
(2097152, 24, 2) (2097152, 24)
(2097152, 2, 2)
Using deposition function: anisotropic_2d
Total mass error on cosmo:	 0.73 %
--------------------------------------------------
Method: anisotropic 3D
(1000000, 24, 3) (1000000, 24)
(1000000, 3, 3)


: 

In [ ]:
with open(f'../../../data/dataset_cosmo_2d.pkl', 'rb') as f:
    data = pickle.load(f)
pos = data['pos'][:]
mass = data['mass'][:, np.newaxis]
boxsize = data['boxsize']
extent = [0, boxsize]

# sample the particles for visualization purposes
N = 1000
pos = pos[::N]
mass = mass[::N]

hsm_tensor, hsm_eigvals, hsm_eigvecs = compute_hsm_tensor(
    pos,
    mass,
    NN=16,
    boxsize=boxsize
    )

hsm_circular = compute_hsm(
    pos, 16, boxsize=boxsize
    )[0]
hsm_tensor, hsm_eigvals, hsm_eigvecs = project_hsm_tensor_to_2d(hsm_tensor, plane=(0, 1))

In [ ]:
print(hsm_eigvals.max())
print(hsm_circular.max())

In [ ]:
from matplotlib.patches import Circle, Ellipse

N = 10
fig, ax = plt.subplots(figsize=(3, 3))

def draw_circle(ax, center, radius):
    circle = Circle(center, radius, 
                    fill=False, edgecolor='k', lw=0.5, alpha=1.0)
    ax.add_patch(circle)

def draw_ellipse(ax, center, width, height, angle):
    ellipse = Ellipse(center, width, height, angle=angle, 
                      fill=False, edgecolor='k', lw=0.5, alpha=1.0, ls='--')
    ax.add_patch(ellipse)

print(np.max(hsm_eigvals[::N] / hsm_circular[::N, np.newaxis]))

for p, h, w, v in zip(pos[::N], 
                      hsm_circular[::N], 
                      hsm_eigvals[::N], 
                      hsm_eigvecs[::N]):

    # Calculate width, height, and angle of the ellipse
    width, height = w
    angle = np.degrees(np.arctan2(*v[:, 0][::-1]))

    #if width / h > 10 or height / h > 10:
    #    m = 'd'
    #else:
    #    m = 'o'
    #ax.scatter(pos[::N, 0], 
    #           pos[::N, 1], 
    #           s=0.5, alpha=0.5, marker=m, color='navy')

    # Draw the smoothing circle and ellipse
    center = (p[0], p[1])
    draw_circle(ax, center, h)
    draw_ellipse(ax, center, width, height, angle)

plt.xlim(-500, 12000+500)
plt.ylim(-500, 12000+500)
plt.xlabel('x [a. u.]')
plt.ylabel('y [a. u.]')
#plt.savefig('../plots/mass_conservation_test')
plt.show()

In [ ]:
raise SystemExit

In [ ]:
# -------------------------------
# Test harness
# -------------------------------
np.random.seed(42)

dim = 3
N = 1000
num_fields = 3
gridnum = 32
extent = np.array([0.0, 1.0])
periodic = True

# random positions inside extent
positions = np.random.uniform(extent[0], extent[1], size=(N, dim)).astype(np.float32)
quantities = np.random.rand(N, num_fields).astype(np.float32)

In [ ]:
grid_py = p2g(positions,
           quantities,
           averaged=[False],
           gridnum=150,
           extent=[extent[0], extent[1]],
           #periodic=0,
           num_nn=16,
           method='ngp',
           accelerator='python'
           )

grid_cp = p2g(positions,
           quantities,
           averaged=[False],
           gridnum=150,
           extent=[extent[0], extent[1]],
           #periodic=0,
           num_nn=16,
           method='ngp',
           accelerator='cpp'
           )

In [ ]:
print(np.allclose( grid_py, grid_cp ))

In [ ]:
import pickle
with open('../../../data/dataset_random_2d.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data.keys()
pos = data['pos'][:]
mass = data['mass'][:, np.newaxis]
boxsize = data['boxsize']
print(pos.min(), pos.max(), mass.min(), mass.max(), boxsize)

In [ ]:
print(type(pos), type(mass) , type(boxsize))
print(pos.shape)

In [ ]:
grid_py = p2g(pos,
           mass,
           averaged=[False],
           gridnum=150,
           extent=[0, boxsize],
           #periodic=0,
           #num_nn=16,
           method='ngp',
           accelerator='python'
           )

In [ ]:
grid_cp = p2g(pos,
           mass,
           averaged=[False],
           gridnum=150,
           extent=[0, boxsize],
           #periodic=0,
           #num_nn=16,
           method='ngp',
           accelerator='cpp'
           )

In [ ]:
print(np.allclose( grid_py, grid_cp ))

In [ ]:
grid_py.shape

In [ ]:
plt.figure()
plt.scatter(pos[:,0], pos[:,1], s=1, alpha=0.1)
plt.show()

In [ ]:
plt.figure()
plt.imshow(grid_cp[..., 0])
plt.show()

In [ ]:
raise SystemExit

In [ ]:
ds = pn.load('../../../data/snapshot_176_128.hdf5')

In [ ]:
# simulation information
periodic = True
boxsize  = 15000
extent   = np.array([0, boxsize])

# grid information
NN       = 8
gridnum  = 50

In [ ]:
# needed fields
pos      = ds.gas['pos']
masses   = ds.gas['mass']
temp     = ds.gas['temp']
metals   = ds.gas['metals'][:, 0]

# IMPORTANT: hsm (tensor) needs to be computed from the 3d distribution (unless the simulation is 2d)!!
hsm = compute_hsm(pos, NN, boxsize)[0]
hmat, eλ, ev = compute_hsm_tensor(pos, masses, NN, boxsize)

# stack all quantities to deposit
quantities = np.stack([masses, metals, temp], axis=-1); print(quantities.shape)

# do we need to average the deposited quantities or not?
averaged   = [False, True, True]

#### 2D case

In [ ]:
mask = np.logical_and(pos[:, -1] > 0,
                      pos[:, -1] < 3000)

pos_masked          = pos[mask][:, :2]
masses_masked       = masses[mask]
hsm_masked          = hsm[mask]
hmat_masked        = hmat[mask]
quantities_masked   = quantities[mask]
hmat_2d, eλ_2d, ev_2d = project_hsm_tensor_to_2d(hmat_masked, plane=(0, 1)) 

kwargs = {'positions': pos_masked, 
          'quantities': quantities_masked, 
          'averaged': averaged, 
          'extent': extent, 
          'gridnum': gridnum, 
          'periodic': 1}

In [ ]:
fields_ngp = p2g(**kwargs, method='ngp')
fields_cic = p2g(**kwargs, method='cic')
fields_tsc = p2g(**kwargs, method='tsc')

fields_ngp_cpp = p2g(**kwargs, method='ngp', accelerator='cpp')
fields_cic_cpp = p2g(**kwargs, method='cic', accelerator='cpp')
fields_tsc_cpp = p2g(**kwargs, method='tsc', accelerator='cpp')

fields_cic_ada_cpp = p2g(**kwargs, method='cic_adaptive', accelerator='cpp', num_nn=NN)
fields_tsc_ada_cpp = p2g(**kwargs, method='tsc_adaptive', accelerator='cpp', num_nn=NN)

fields_sph_iso_cpp_1   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='gaussian')
fields_sph_iso_cpp_2   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='super_gaussian')
fields_sph_iso_cpp_3   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='cubic')
fields_sph_iso_cpp_4   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='quintic')
fields_sph_iso_cpp_5   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='wendland_c2')
fields_sph_iso_cpp_6   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='wendland_c4')
fields_sph_iso_cpp_7   = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='wendland_c6')

fields_sph_aniso_cpp_1 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='gaussian')
fields_sph_aniso_cpp_2 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='super_gaussian')
fields_sph_aniso_cpp_3 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='cubic')
fields_sph_aniso_cpp_4 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='quintic')
fields_sph_aniso_cpp_5 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='wendland_c2')
fields_sph_aniso_cpp_6 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='wendland_c4')
fields_sph_aniso_cpp_7 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvals=eλ_2d, hmat_eigvecs=ev_2d, kernel_name='wendland_c6')

In [ ]:
fig, ax = plt.subplots(8, quantities.shape[1], figsize=(16, 72))
for i in range(quantities.shape[1]):
    ax[0, i].imshow(np.log10(fields_ngp)[:, :, i])

for i in range(quantities.shape[1]):
    ax[1, i].imshow(np.log10(fields_cic)[:, :, i])

for i in range(quantities.shape[1]):
    ax[2, i].imshow(np.log10(fields_tsc)[:, :, i])

for i in range(quantities.shape[1]):
    ax[3, i].imshow(np.log10(fields_ngp_cpp)[:, :, i])

for i in range(quantities.shape[1]):
    ax[4, i].imshow(np.log10(fields_cic_cpp)[:, :, i])

for i in range(quantities.shape[1]):
    ax[5, i].imshow(np.log10(fields_cic_ada_cpp)[:, :, i])

for i in range(quantities.shape[1]):
    ax[6, i].imshow(np.log10(fields_tsc_cpp)[:, :, i])

for i in range(quantities.shape[1]):
    ax[7, i].imshow(np.log10(fields_tsc_ada_cpp)[:, :, i])
plt.show()

In [ ]:
fig, ax = plt.subplots(14, quantities.shape[1], figsize=(16, 72))
for i in range(quantities.shape[1]):
    ax[0, i].imshow(np.log10(fields_sph_iso_cpp_1)[:, :, i])

for i in range(quantities.shape[1]):
    ax[1, i].imshow(np.log10(fields_sph_iso_cpp_2)[:, :, i])

for i in range(quantities.shape[1]):
    ax[2, i].imshow(np.log10(fields_sph_iso_cpp_3)[:, :, i])

for i in range(quantities.shape[1]):
    ax[3, i].imshow(np.log10(fields_sph_iso_cpp_4)[:, :, i])

for i in range(quantities.shape[1]):
    ax[4, i].imshow(np.log10(fields_sph_iso_cpp_5)[:, :, i])

for i in range(quantities.shape[1]):
    ax[5, i].imshow(np.log10(fields_sph_iso_cpp_6)[:, :, i])

for i in range(quantities.shape[1]):
    ax[6, i].imshow(np.log10(fields_sph_iso_cpp_7)[:, :, i])


for i in range(quantities.shape[1]):
    ax[7, i].imshow(np.log10(fields_sph_aniso_cpp_1)[:, :, i])

for i in range(quantities.shape[1]):
    ax[8, i].imshow(np.log10(fields_sph_aniso_cpp_2)[:, :, i])

for i in range(quantities.shape[1]):
    ax[9, i].imshow(np.log10(fields_sph_aniso_cpp_3)[:, :, i])

for i in range(quantities.shape[1]):
    ax[10, i].imshow(np.log10(fields_sph_aniso_cpp_4)[:, :, i])

for i in range(quantities.shape[1]):
    ax[11, i].imshow(np.log10(fields_sph_aniso_cpp_5)[:, :, i])

for i in range(quantities.shape[1]):
    ax[12, i].imshow(np.log10(fields_sph_aniso_cpp_6)[:, :, i])

for i in range(quantities.shape[1]):
    ax[13, i].imshow(np.log10(fields_sph_aniso_cpp_7)[:, :, i])
plt.show()

#### 3D case

In [ ]:
kwargs = {'positions': pos, 
          'quantities': quantities, 
          'averaged': averaged, 
          'extent': extent, 
          'gridnum': gridnum, 
          'periodic': 1}

In [ ]:
fields_ngp = p2g(**kwargs, method='ngp')
fields_cic = p2g(**kwargs, method='cic')
fields_tsc = p2g(**kwargs, method='tsc')

fields_ngp_cpp = p2g(**kwargs, method='ngp', accelerator='cpp')
fields_cic_cpp = p2g(**kwargs, method='cic', accelerator='cpp')
fields_tsc_cpp = p2g(**kwargs, method='tsc', accelerator='cpp')

fields_cic_ada_cpp = p2g(**kwargs, method='cic_adaptive', accelerator='cpp', num_nn=NN)
fields_tsc_ada_cpp = p2g(**kwargs, method='tsc_adaptive', accelerator='cpp', num_nn=NN)

fields_sph_iso_cpp_1 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='gaussian')
fields_sph_iso_cpp_2 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='super_gaussian')
fields_sph_iso_cpp_3 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='cubic')
fields_sph_iso_cpp_4 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='quintic')
fields_sph_iso_cpp_5 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='wendland_c2')
fields_sph_iso_cpp_6 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='wendland_c4')
fields_sph_iso_cpp_7 = p2g(**kwargs, method='sph_isotropic', accelerator='cpp', hsm=hsm, kernel_name='wendland_c6')

fields_sph_aniso_cpp_1 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='gaussian')
fields_sph_aniso_cpp_2 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='super_gaussian')
fields_sph_aniso_cpp_3 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='cubic')
fields_sph_aniso_cpp_4 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='quintic')
fields_sph_aniso_cpp_5 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='wendland_c2')
fields_sph_aniso_cpp_6 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='wendland_c4')
fields_sph_aniso_cpp_7 = p2g(**kwargs, method='sph_anisotropic', accelerator='cpp', hmat_eigvecs=ev, hmat_eigvals=eλ, kernel_name='wendland_c6')

In [ ]:
fig, ax = plt.subplots(8, quantities.shape[1], figsize=(16, 72))
for i in range(quantities.shape[1]):
    ax[0, i].imshow(np.log10(fields_ngp.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[1, i].imshow(np.log10(fields_cic.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[2, i].imshow(np.log10(fields_tsc.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[3, i].imshow(np.log10(fields_ngp_cpp.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[4, i].imshow(np.log10(fields_cic_cpp.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[5, i].imshow(np.log10(fields_cic_ada_cpp.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[6, i].imshow(np.log10(fields_tsc_cpp.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[7, i].imshow(np.log10(fields_tsc_ada_cpp.mean(axis=0))[:, :, i])

plt.show()

In [ ]:
fig, ax = plt.subplots(14, quantities.shape[1], figsize=(16, 72))
for i in range(quantities.shape[1]):
    ax[0, i].imshow(np.log10(fields_sph_iso_cpp_1.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[1, i].imshow(np.log10(fields_sph_iso_cpp_2.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[2, i].imshow(np.log10(fields_sph_iso_cpp_3.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[3, i].imshow(np.log10(fields_sph_iso_cpp_4.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[4, i].imshow(np.log10(fields_sph_iso_cpp_5.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[5, i].imshow(np.log10(fields_sph_iso_cpp_6.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[6, i].imshow(np.log10(fields_sph_iso_cpp_7.mean(axis=0))[:, :, i])


for i in range(quantities.shape[1]):
    ax[7, i].imshow(np.log10(fields_sph_aniso_cpp_1.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[8, i].imshow(np.log10(fields_sph_aniso_cpp_2.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[9, i].imshow(np.log10(fields_sph_aniso_cpp_3.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[10, i].imshow(np.log10(fields_sph_aniso_cpp_4.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[11, i].imshow(np.log10(fields_sph_aniso_cpp_5.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[12, i].imshow(np.log10(fields_sph_aniso_cpp_6.mean(axis=0))[:, :, i])

for i in range(quantities.shape[1]):
    ax[13, i].imshow(np.log10(fields_sph_aniso_cpp_7.mean(axis=0))[:, :, i])
plt.show()

In [ ]:
raise SystemExit

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(12, 9))

for j in range(4):
    fov = np.s_[50:150, -100:]

    ax[j, 0].imshow(np.log10(fields[j][fov][...,0]), cmap='bone', vmin=-4)
    ax[j, 1].imshow(fields[j][fov][...,1], cmap='twilight')
    ax[j, 2].imshow(np.log10(fields[j][fov][...,-2]), cmap='binary_r')
    ax[j, 3].imshow(np.log10(fields[j][fov][...,-1]), cmap='gist_heat')

for a in ax.flat:
    a.axis('off')

plt.tight_layout()
fig.savefig('plots/deposition_comparison.png', transparent=True, dpi=300)
plt.show()

We can plot the smoothing ellipses (projected 3d-ellipsoids to the plane) compared to the isotropic smoothing lengths (circles)

In [ ]:
from matplotlib.patches import Circle, Ellipse

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(pos[:, 0], pos[:, 1], s=1, ec='none', alpha=0.5)

def draw_circle(ax, center, radius):
    circle = Circle(center, radius, 
                    fill=False, edgecolor='k', lw=1, alpha=0.5)
    ax.add_patch(circle)

def draw_ellipse(ax, center, width, height, angle):
    ellipse = Ellipse(center, width, height, angle=angle, 
                      fill=False, edgecolor='g', lw=1, alpha=0.8)
    ax.add_patch(ellipse)


for p, h, w, v in zip(pos[::250], 
                      hsm[::250], 
                      eλ_2d[::250], 
                      ev_2d[::250]):

    # Calculate width, height, and angle of the ellipse
    width, height = 2*w
    angle = np.degrees(np.arctan2(*v[:, 0][::-1]))

    # Draw the smoothing circle and ellipse
    center = (p[0], p[1])
    draw_circle(ax, center, h)
    draw_ellipse(ax, center, width, height, angle)
    